# Predicting algorithm time

In [ ]:
import statistics

epoch_count_1=5
epoch_count_2=5

phase_1_elapsed_ms = [48,53,35,34,33,33,32,31,33,32] # ms are obtained from training notebook, combination MobileNetV2/merged_datasets_k16-NORM
phase_2_elapsed_ms = [60, 69, 61, 63, 63, 64, 60, 60, 64, 63]
sec_per_batch_phase_1 = statistics.mean(phase_1_elapsed_ms[:epoch_count_1])/1000
sec_per_batch_phase_2 = statistics.mean(phase_2_elapsed_ms[:epoch_count_2])/1000

def printTrainTime(total_batch_count):
  print(f"{(((total_batch_count*(((sec_per_batch_phase_1)*epoch_count_1)+((sec_per_batch_phase_2)*epoch_count_2)))/60)/60):.2f} h")

## Exection time for Version 2 - test shared inside single division

In [ ]:
# dataset_sm_count = 5000 - 2 * 625
N_min = 1
balanced_size = 9504*9

balanced_batch_count = balanced_size//32 + (balanced_size % 32 > 0)


K = [
    k
    for len in range(balanced_size)
    for k in range(40)
    if (k + 0.2) * len == balanced_size
]

iterations_max = N_min * K[0]


print(f"Batches in total dataset {balanced_size//32}")
print(f"Factors of {balanced_size}, >= {K[0]} = |{len(K)}|:\t {K}\n")

batch_number=0

for k in K:

    iterations=iterations_max/k # Koliko puta trebamo iterirati kroz citavi dataset?
    batches=balanced_batch_count*iterations
    batch_number+=batches

    print(f"""\
    k = {k:2.0f} => ({k:2.0f} + 0.2) x {(balanced_size//(k+0.2)):5.0f} = {k*(balanced_size//k)}\
    Iterations: {(iterations_max//k):2.0f}\tExtra runs: {(iterations_max%k):2.0f}\t Batches: {batches:5.1f}\
    """)



print("Total batches to fit: ",batch_number//1)

printTrainTime(batch_number)

Batches in total dataset 2673
Factors of 85536, >= 35 = |8|:	 [35, 19, 16, 13, 7, 3, 2, 1]

    k = 35 => (35 + 0.2) x  2429 = 85505    Iterations:  1	Extra runs:  0	 Batches: 2673.0    
    k = 19 => (19 + 0.2) x  4455 = 85519    Iterations:  1	Extra runs: 16	 Batches: 4923.9    
    k = 16 => (16 + 0.2) x  5280 = 85536    Iterations:  2	Extra runs:  3	 Batches: 5847.2    
    k = 13 => (13 + 0.2) x  6480 = 85527    Iterations:  2	Extra runs:  9	 Batches: 7196.5    
    k =  7 => ( 7 + 0.2) x 11879 = 85533    Iterations:  5	Extra runs:  0	 Batches: 13365.0    
    k =  3 => ( 3 + 0.2) x 26729 = 85536    Iterations: 11	Extra runs:  2	 Batches: 31185.0    
    k =  2 => ( 2 + 0.2) x 38879 = 85536    Iterations: 17	Extra runs:  1	 Batches: 46777.5    
    k =  1 => ( 1 + 0.2) x 71280 = 85536    Iterations: 35	Extra runs:  0	 Batches: 93555.0    
Total batches to fit:  205523.0
29.63 h


## Execution time for Version 3 - separate test for each partition

In [ ]:
import statistics

epoch_count_1=5
epoch_count_2=5

phase_1_elapsed_ms = [48,53,35,34,33,33,32,31,33,32]
phase_2_elapsed_ms = [60, 69, 61, 63, 63, 64, 60, 60, 64, 63]
sec_per_batch_phase_1 = statistics.mean(phase_1_elapsed_ms[:epoch_count_1])/1000
sec_per_batch_phase_2 = statistics.mean(phase_2_elapsed_ms[:epoch_count_2])/1000

get_all_divisors_less_than = (
    lambda k_max, n: sorted(set([
        i for d in (
            [i]
            for i in range(1, min(int(k_max**0.5) + 1, n + 1))
            if k_max % i == 0
            )
            for i in d
        ]), reverse = True)
)

dataset_sm_count = 5000 - 2 * 625
N_min = 1
da_balanced_count = 9504*9

da_balanced_batch_count = da_balanced_count//32 + (da_balanced_count % 32 > 0)
k_max = da_balanced_count//dataset_sm_count
K = get_all_divisors_less_than(da_balanced_count,k_max)
iterations_max = N_min*K[0]

print(f"Batches in total dataset {da_balanced_batch_count}")
print(f"k_max = round({da_balanced_count} / {dataset_sm_count}) = {k_max}")
print(f"Factors of {da_balanced_count}, >= {k_max} = |{len(K)}|:\t", K, "\n")

batch_number=0

for k in K:
    iterations=iterations_max/k # Koliko puta trebamo iterirati kroz citavi dataset?
    batches=da_balanced_batch_count*iterations
    batch_number+=batches
    print(f"""\
    k = {k:2.0f} => {k:2.0f} x {(da_balanced_count//k):5.0f} = {k*(da_balanced_count//k)}\
    Iterations: {(iterations_max//k):2.0f}\tExtra runs: {(iterations_max%k):2.0f}\t Batches: {batches:5.1f}\
    """)

print("Total batches to fit: ",batch_number//1)

printTrainTime(batch_number)

Batches in total dataset 2673
k_max = round(85536 / 3750) = 22
Factors of 85536, >= 22 = |12|:	 [22, 18, 16, 12, 11, 9, 8, 6, 4, 3, 2, 1] 

    k = 22 => 22 x  3888 = 85536    Iterations:  1	Extra runs:  0	 Batches: 2673.0    
    k = 18 => 18 x  4752 = 85536    Iterations:  1	Extra runs:  4	 Batches: 3267.0    
    k = 16 => 16 x  5346 = 85536    Iterations:  1	Extra runs:  6	 Batches: 3675.4    
    k = 12 => 12 x  7128 = 85536    Iterations:  1	Extra runs: 10	 Batches: 4900.5    
    k = 11 => 11 x  7776 = 85536    Iterations:  2	Extra runs:  0	 Batches: 5346.0    
    k =  9 =>  9 x  9504 = 85536    Iterations:  2	Extra runs:  4	 Batches: 6534.0    
    k =  8 =>  8 x 10692 = 85536    Iterations:  2	Extra runs:  6	 Batches: 7350.8    
    k =  6 =>  6 x 14256 = 85536    Iterations:  3	Extra runs:  4	 Batches: 9801.0    
    k =  4 =>  4 x 21384 = 85536    Iterations:  5	Extra runs:  2	 Batches: 14701.5    
    k =  3 =>  3 x 28512 = 85536    Iterations:  7	Extra runs:  1	 Batches: 